## Utils for plots

In [ ]:
def save_fig(test_var,*xtra):#,formats,**xtra1
    if save_figs:
        
        base_folder='Plots'
        file_name=time.strftime("%Y%m%d") +'_'+ test_var
        if save_inh_names:
            inhibitors=''.join(inh_order)
            file_name=file_name+'_'+inhibitors
        plot_path_png=f'{base_folder}\\{results_folder}\\{file_name}'
        plot_path_svg=plot_path_png
    
        for stuff in xtra:
            file_name=file_name+'_'+stuff
    
    
        if plot_formats=='b':
            plt.savefig(plot_path_png+'.png',bbox_inches='tight', dpi=300)
            plt.savefig(plot_path_svg+'.svg',bbox_inches='tight')
    
        if plot_formats=='s':
            plt.savefig(plot_path_svg+'.svg',bbox_inches='tight')
    
        else:
            plt.savefig(plot_path_png+'.png',bbox_inches='tight', dpi=300)
    
    
    
    #plt.savefig(results_folder + f'\\heatmap_zcount_{var}_inside{inside}_{orient}.png',bbox_inches='tight', dpi=300)
    #test_var='count'
    #xtra=f'inside{inside}_orient{orient}_filter{im_filter}'
    #save_fig(test_var,'both',xtra)


# Rolling Means

In [1]:
#---------------------------------------------------------------------------
#Function for count plots with rolling mean


def rolling_mean(df,grouping_var,y_var,x_var):
    dv_=[]
    #x_var=['time']
    if grouping_var=='inh':
        grouping_vars=[grouping_var]+['inh_exp_id']
    else:
        grouping_vars=[grouping_var]+['inh','inh_exp_id']
    grouping1=grouping_vars+x_var
    if x_var==['time']:
        
        print(grouping1)
    else:
        grouping1=grouping1+['time']
        print(grouping1)
    
    #vars_to_keep=['inh','inh_exp_id','time']
    #for xtra in extra_vars:
    #    vars_to_keep+=xtra
    #Grupperar först på inh_exp_id och tar medel för varje experiment
    df_grouped=df.groupby(grouping1).mean()[[y_var]]#.rename(columns={'pid':'plts'})#
    df_grouped=df_grouped.reset_index()
    #Grupper återigen resultatet av förra grupperingen, denna gång utan tid i grupperingsvariabeln 
    dfg=df_grouped.groupby(grouping_vars)
    
    #Kör rolling på varje enskild grupp
    for i,gr in dfg:
    
        df2=gr[[y_var]].rolling(window=8,win_type='blackman',min_periods=3,center=True).mean()#.\'bartlett'
                #rename(columns={'pid':f'{grouping_var}/{x_var}_roll'})
        dv_.append(df2)

    dv=pd.concat(dv_, axis=0)
    df_grouped['roll']=dv#*40
    #df_grouped['pid']=dv#*40#df_grouped[f'{grouping_var}_roll']=dv*40
    #dv=dv.reset_index()
    
    
    #df=dfg.reset_index()
    #df=pd.concat((df,dv),axis=1)
    #dv=dv.sort_values('').set_index('id')
    #df=pd.concat((df,dv),axis=1)
    return df_grouped

#---------------------------------------------------------------------------

In [ ]:
def rolling_bartlett(gr):
    gr[f'{y_var}_roll']=gr[y_var].rolling(window=6,win_type='bartlett',min_periods=3,center=True).mean()
    gr['count_roll']=gr['count'].rolling(window=4,win_type='bartlett',min_periods=3,center=True).mean()
    return gr

In [ ]:
def rolling_mean_zled(dfg,y_var):
    
#y_var='c0_mean'
#count_threshold=10
    dfg=df.groupby(['inside_injury','inh','zled','time']).mean().reset_index()#Lägg till inh_exp_id senare!
    df_count=df.groupby(['inside_injury','inh','zled','time']).count()['pid'].reset_index()#Lägg till inh_exp_id senare!
    dfg['count']=df_count['pid']
    dfg=dfg.groupby(['inside_injury','inh','zled']).filter(lambda x:x['count'].sum() >1000)
#dfg.loc[((dfg['count']<count_threshold)),:]=np.nan
    dfg=dfg.groupby(['inside_injury','inh','zled']).apply(rolling_bartlett)
    dfg=dfg[['inside_injury','inh','zled','time',y_var,f'{y_var}_roll','count']]
    dfg=dfg.reset_index()
    return dfg

In [1]:
#---------------------------------------------------------------------------
#Function for count plots with rolling mean


def rolling_count(df,grouping_var,x_var):
    dv_=[]
    dg_=[]
    #x_var=['time']
    grouping_vars=grouping_var+['inh','inh_exp_id']
    grouping1=grouping_vars+x_var
    if x_var==['time']:
        
        print(grouping1)
    else:
        grouping1=grouping1+['time']
        print(grouping1)
    
    #vars_to_keep=['inh','inh_exp_id','time']
    #for xtra in extra_vars:
    #    vars_to_keep+=xtra
    df_grouped=df.groupby(grouping1).count()[['pid']]#.rename(columns={'pid':'plts'})#
  
    
    df_grouped=df_grouped.reset_index()
    dfg=df_grouped.groupby(grouping_vars)
    
    
    for i,gr in dfg:
    
        df2=gr[['pid']].rolling(window=6,win_type='bartlett',min_periods=3,center=True).mean()#.\
                #rename(columns={'pid':f'{grouping_var}/{x_var}_roll'})
        dv_.append(df2)
        df_gr=df2.diff()#.rolling(window=4,win_type='bartlett',min_periods=2,center=True).mean()
        dg_.append(df_gr)
        

    dv=pd.concat(dv_, axis=0)
    df_grouped['roll']=dv*40
    d_gr=pd.concat(dg_, axis=0)
    df_grouped['diff']=d_gr*40
    
    #df_grouped['pid']=dv#*40#df_grouped[f'{grouping_var}_roll']=dv*40
    #dv=dv.reset_index()
    if x_var==['minute']:
        grouping2=grouping_vars+x_var
        #grouping3=grouping2+['time']
        #df_grouped=df_grouped.groupby(grouping3).count()[['pid']].reset_index()#.rename(columns={'pid':'plts'})#
        df_grouped=df_grouped.groupby(grouping2).mean()[['roll']].reset_index()
    
    
    #df=dfg.reset_index()
    #df=pd.concat((df,dv),axis=1)
    #dv=dv.sort_values('').set_index('id')
    #df=pd.concat((df,dv),axis=1)
    return df_grouped

#---------------------------------------------------------------------------

In [ ]:
#---------------------------------------------------------------------------
#Function for percentile plots with rolling mean


def rolling_perc(df1,df2,grouping_var,x_var):
    dv_=[]
    #x_var=['time']
    grouping_vars=grouping_var+['inh','inh_exp_id']
    grouping1=grouping_vars+x_var
    if x_var==['time']:
        
        print(grouping1)
    else:
        grouping1=grouping1+['time']
        print(grouping1)
    
    #vars_to_keep=['inh','inh_exp_id','time']
    #for xtra in extra_vars:
    #    vars_to_keep+=xtra
    
    dfg1=df1.groupby(grouping1).count()['pid']#.reset_index()
    dfg2=df2.groupby(grouping1).count()['pid']
    dfg=dfg2/dfg1
    df_grouped=dfg.reset_index()
    
    dfg=df_grouped.groupby(grouping_vars)
    
    
    for i,gr in dfg:
    
        df2=gr[['pid']].rolling(window=8,win_type='blackman',min_periods=3,center=True).mean()
        #.rolling(window=6,win_type='bartlett',min_periods=3,center=True).mean()#.\
                #rename(columns={'pid':f'{grouping_var}/{x_var}_roll'})
        dv_.append(df2)

    dv=pd.concat(dv_, axis=0)
    df_grouped['roll']=dv#*40
    #df_grouped['pid']=dv#*40#df_grouped[f'{grouping_var}_roll']=dv*40
    #dv=dv.reset_index()
    if x_var==['minute']:
        grouping2=grouping_vars+x_var
        #grouping3=grouping2+['time']
        #df_grouped=df_grouped.groupby(grouping3).count()[['pid']].reset_index()#.rename(columns={'pid':'plts'})#
        df_grouped=df_grouped.groupby(grouping2).mean()[['roll']].reset_index()
    
    
    #df=dfg.reset_index()
    #df=pd.concat((df,dv),axis=1)
    #dv=dv.sort_values('').set_index('id')
    #df=pd.concat((df,dv),axis=1)
    return df_grouped

#---------------------------------------------------------------------------

## Detection and visualization of outliers

In [1]:
def outliers_nrtracks(pc,test_var):
    pc_test=pc#[(pc.tracked=='True')]
    #df['inh_id']=df['inh'].astype('str')+'-'+df['inh_exp_id'].astype('int').astype('str')
    #dfg=pc_test.groupby(['inh','inh_id']).mean()[[test_var]].reset_index()#.set_index(
    dfg=pc_test.groupby(['inh','path']).mean()[[test_var]].reset_index()
        #['inh',test_var]).sort_index().reset_index()
    outliers=[]
    for inh in dfg.inh.unique():
        dfg_inh=dfg[(dfg.inh==inh)].copy()
        outliers.append(dfg_inh[(np.abs(stats.zscore(dfg_inh[test_var])) > 2)]['path'])#['inh_id'])
    df_outliers=pd.concat(outliers,axis=0)
    dfg['outlier']=dfg.path.isin(df_outliers)
    g=sns.catplot(data=dfg,y=test_var,x='inh',hue='outlier',height=5,aspect=3,kind='swarm',legend=False)
    #if save_figs:
        
        
    return dfg[(dfg.outlier==True)]

In [ ]:
# Visualize distribution of observation number in individual experiments
def outliers_count(df):
    hue_order=['True','False','Both']
    df['tracked']=df.nrtracks>1
    #df['tracked']=df.tracked.astype('str').copy()
    dfg_count=df.groupby(['inh','exp_id','tracked']).count()[['pid']].reset_index()#.set_index(#,'mouse','inj',
       # ['inh','pid']).sort_index().reset_index()
    dfg_count1=df.groupby(['inh','exp_id']).count()[['pid']].reset_index().set_index(
        ['inh','pid']).sort_index().reset_index()#,'mouse','inj'
    dfg_count1['tracked']='Both'
    dfg=pd.concat([dfg_count,dfg_count1],axis=0)
    outliers=[]
    for inh in dfg.inh.unique():
        dfg_inh=dfg[(dfg.inh==inh)&(dfg.tracked=='True')].copy()
        outliers.append(dfg_inh[(np.abs(stats.zscore(dfg_inh.pid)) > 2)]['exp_id'])
    df_outliers=pd.concat(outliers,axis=0)
    dfg['outlier']=dfg.exp_id.isin(df_outliers)
     
        #dfg_count=df.groupby(['inh','exp_id','mouse','inj','tracked']).count()[['pid']].reset_index().set_index(
        #['inh','pid']).sort_index().reset_index()
    #print(dfg_count.head(5))
    # Plot visualising distribution of observations of tracked platelets
    #with sns.axes_style("whitegrid"):
    g=sns.catplot(data=dfg,y="pid",x='tracked',col='inh',hue='outlier',col_wrap=3,height=4,kind='swarm')
    return dfg[(dfg.outlier==True)&(dfg.tracked=='True')]
    #sns.despine(bottom=True)
    #dodge=True, ,
    #g = sns.FacetGrid(data=dfg_count,hue="tracked",x="inh", palette=pal_many_c)
    #g = (g.map(sns.stripplot,y="pid"))
    #hue="tracked",hue_order=hue_order,dodge=True,

In [ ]:
# Visualize distribution of observation number in individual experiments
def outliers_count1(df):
    
    dfg=df.groupby(['inh','path']).count()[['pid']].reset_index()#.set_index(#,'mouse','inj',
       # ['inh','pid']).sort_index().reset_index()
    outliers=[]
    for inh in dfg.inh.unique():
        dfg_inh=dfg[(dfg.inh==inh)].copy()
        outliers.append(dfg_inh[(np.abs(stats.zscore(dfg_inh.pid)) > 2)]['path'])
    df_outliers=pd.concat(outliers,axis=0)
    dfg['outlier']=dfg.path.isin(df_outliers)
     
        #dfg_count=df.groupby(['inh','exp_id','mouse','inj','tracked']).count()[['pid']].reset_index().set_index(
        #['inh','pid']).sort_index().reset_index()
    #print(dfg_count.head(5))
    # Plot visualising distribution of observations of tracked platelets
    #with sns.axes_style("whitegrid"):
    g=sns.catplot(data=dfg,y="pid",x='inh',hue='outlier',height=5,aspect=3,kind='swarm')
    return dfg[(dfg.outlier==True)]
    #sns.despine(bottom=True)
    #dodge=True, ,
    #g = sns.FacetGrid(data=dfg_count,hue="tracked",x="inh", palette=pal_many_c)
    #g = (g.map(sns.stripplot,y="pid"))
    #hue="tracked",hue_order=hue_order,dodge=True,

In [ ]:
def outliers_fluo(pc):
    pc_test=pc[(pc.tracked==True)]#(pc.inside_injury==True)&(pc.height=='bottom')&
    dfg_fluo=pc_test.groupby(['inh','path']).mean()[['c0_mean','c0_max','c2_mean','c1_mean','c1_max']].reset_index()#.set_index(
        #['inh','exp_id']).sort_index().reset_index()#,'mouse','inj'
    id_cols=['inh','path',]#'mouse','inj',
    melt_vars=['c0_mean','c0_max','c2_mean','c1_mean','c1_max']
    ['c0_mean','c0_max','c2_mean','c1_mean','c1_max']
    dfg_fluo_long=dfg_fluo.melt(id_vars=id_cols,value_vars=melt_vars,var_name='Measure',value_name='Fluorescence')
    outliers=[]
    for inh in dfg_fluo_long.inh.unique():
            for measure in dfg_fluo_long.Measure.unique():
                dfg_=dfg_fluo_long[(dfg_fluo_long.inh==inh)&(dfg_fluo_long.Measure==measure)].copy()
                dfg_['outlier']=(np.abs(stats.zscore(dfg_.Fluorescence)) > 2)
                outliers.append(dfg_)
    
    try:
        df_outliers=pd.concat(outliers,axis=0)
    except ValueError: 
        print(pc_test)
    
    g=sns.catplot(data=df_outliers,y="Fluorescence",x='inh',hue='outlier',row='Measure',height=5,aspect=3,kind='swarm',sharey=False)
    #dfg_fluo_long['outlier']=dfg_fluo_long.exp_id.isin(df_outliers)
#dfg_fluo_long.head()
    
    return df_outliers[df_outliers.outlier==True]

# Visualize distribution of observations in individual experiments
def outliers_count(df):
    hue_order=['True','False','Both']
    df['tracked']=df.tracked.astype('str').copy()
    dfg_count=df.groupby(['inh','exp_id''tracked']).count()[['pid']].reset_index().set_index(
        ['inh','pid']).sort_index().reset_index()#,'mouse','inj',
    dfg_count1=df.groupby(['inh','exp_id']).count()[['pid']].reset_index().set_index(
        ['inh','pid']).sort_index().reset_index()#,'mouse','inj'
    dfg_count1['tracked']='Both'
    dfg=pd.concat([dfg_count,dfg_count1],axis=0)
    outliers=[]
    for inh in dfg.inh.unique():
        dfg_inh=dfg[(dfg.inh==inh)&(dfg.tracked=='True')].copy()
        outliers.append(dfg_inh[(np.abs(stats.zscore(dfg_inh.pid)) > 2)]['exp_id'])
    df_outliers=pd.concat(outliers,axis=0)
    dfg['outlier']=dfg.exp_id.isin(df_outliers)
     
        #dfg_count=df.groupby(['inh','exp_id','mouse','inj','tracked']).count()[['pid']].reset_index().set_index(
        #['inh','pid']).sort_index().reset_index()
    #print(dfg_count.head(5))
    # Plot visualising distribution of observations of tracked platelets
    #with sns.axes_style("whitegrid"):
    g=sns.catplot(data=dfg,y="pid",x='tracked',col='inh',hue='outlier',col_wrap=3,height=4,kind='swarm')
    return dfg[(dfg.outlier==True)&(dfg.tracked=='True')]

## Mapping positions

In [1]:
#Platelet mapping graph
def plt_map(df_obj,col_var,x_var,vmin,vmax):
    plt.rcParams['image.cmap'] = "turbo"
    sns.set_style("white")
    #plt.rcParams['image.cmap'] = 'jet_r'

#Gor lista av dimensioner 
    lims=['x_s', 'ys', 'zs']
    limsv={}
#Anger gransvarden for de olika dimensionerna genom att kolla minsta och storsta varde och plussa pa 40
    border=1.5
    for l in lims:
        limsv[l]=df_obj[l].min()-20, df_obj[l].max()+20
    
    col='path'
    row='c'
    hue='c'
    x='frame'
    y='z'
    unit='c'

    #Tar ut var 20:e frame fran frame 2 och framat
    frames=pd.unique(df_obj.frame)[::20]+1
    ncols=3
    nrows=len(frames)

    #Gor en lagom stor figur
    
    plt.figure(figsize=(ncols*4,nrows*3))

    #Valjer dimensioner att plotta 
    cols=[('x_s', 'ys'), ('x_s', 'zs'), ('ys', 'zs')]

    ### name väljer variabel att färgsätta plottarna med
    #name='cld'#name='stab'#name='c'#name='depth' #colorv=[1,2,4,8] #name='c2_max'
    name=col_var

    #vmin=0 #vmax=30#vmin=0
    #vmax=10 #vmax=400

    for r, f in enumerate(frames):
        sel_f=df_obj[df_obj.frame==f]
    
        for c, xy in enumerate(cols):
            ax=plt.subplot2grid((nrows, ncols), (r, c))
            #plt.setp()
            ax.scatter(sel_f[xy[0]], sel_f[xy[1]], alpha=0.5, c=sel_f[name],vmin=vmin,vmax=vmax,s=20, linewidth=0.1 )#TOG BORT , vmin=vmin, vmax=vmax,)
            ax.set_title('Time (sec): '+ str(np.round(sel_f[x_var].mean())),fontsize=12)#sel_f.time.mean()
            ax.set_ylim(limsv[xy[1]])
            ax.set_xlim(limsv[xy[0]])
            plt.xticks([])#-37.5,37.5
            plt.yticks([])#-37.5,37.5
            ax.set_xlabel('')
            ax.set_ylabel('')
            ax.tick_params(labelsize=12)
            sns.despine(top=True, right=True, left=True, bottom=True)
            #ax.ticklabel_format()
            #ax.set_axis_bgcolor('black')
    plt.tight_layout()

# Mapping trajectories

In [5]:

def t_traj_mov(df,c_var,vmin,vmax):
    from matplotlib.gridspec import GridSpec
    df=df.sort_values(by=['pid'])

    sns.set_style("white")#sns.set_style("dark")# sns.set_style("white")
    plt.rcParams['image.cmap'] = 'coolwarm'
    plt.rcParams.update({'font.size': 22})
    cols=3
    nrows=3


    
    plot=mov_class_order1

    #c_var='t_tracknr'#'c1_mean'#'tracknr'
    #vmax=50#300
    #vmin=0#100


    

    for c, inhibitor in enumerate(pd.unique(df.inh),0):
        #df=pc_tr[pc_tr.inh==inhibitor]
        #p_list=pd.unique(df[(df.nrtracks>20)].particle)#&(df.height=='bottom')
        #p_df=df[df.particle.isin(p_list)]
        
        fig=plt.figure(figsize=(12,14))#figsize=(4,30)
        #fig.set_title(inhibitor)
        #nrframes=100
    
        #time_bins=int(np.round(190/nrows,0))

        gs=GridSpec(nrows,cols)
        plot_nr=0
        for time in df.time_bin.unique():

    #for frames1,frames2 in zip(np.arange(0,180,time_bins),np.arange(time_bins,180+time_bins,time_bins)):
        #time1=p_df[(p_df.frame>frames1)&(p_df.frame<frames2)]
            for pop in plot:
                plt_pop=df[(df.mov_time==pop)&(df.time_bin==time)&(df.inh==inhibitor)].copy()
                #pop_part=plt_pop.particle.unique()
                ax=fig.add_subplot(gs[plot_nr])
                
                plt_pop.sort_values(by=[c_var],ascending=False)
                plt.scatter(x=plt_pop.x_s, y=plt_pop.ys , c=plt_pop[c_var], s=10, alpha=0.7, cmap='coolwarm',vmax=vmax, vmin=vmin, linewidth=0)
                #plt.scatter(x=0, y=0, s=375,c='none', alpha=0.5, linewidth=40,edgecolor='black')#c='black',     
                circle=plt.Circle((0, 0), 37.5, alpha=0.4,fc='grey',edgecolor='black')#edgecolor='black',linewidth=7,fill='black'
                ax.add_patch(circle)
                plt.axis([-100, 100, -100, 100])
                plt.xlim(-100,100)  
                plt.ylim(-125,100)
                plt.xticks([])#-37.5,37.5
                plt.yticks([])#-37.5,37.5
                if pop==plot[0]:
                    ax.set_ylabel(f'Time range: {np.round(time.left,-1)}-{np.round(time.right,-1)} s',fontsize=14)
                if plot_nr<3:
                    ax.set_title(pop)
                #if pop==plot[2]:
                #    plt.colorbar()
                plot_nr+=1
        sns.despine(top=True, right=True, left=True, bottom=True)
        fig.suptitle(inhibitor, fontsize=16)
        #plt.despine()
            #plt.colorbar
        plt.savefig(results_folder + f'\\t_traj_mov_{c_var}_{plot}.png',bbox_inches='tight', dpi=300)
        plt.show()
    #return
            
            #plt.tight_layout()
    
            

# Heatmaps

In [ ]:
def def_heatmap_params():#max_col,min_col,groups,orient
    import math as m
    

    params_heatmap=dict(xticklabels=15, cmap="turbo",vmax=max_col,vmin=min_col)#,center=center_color)
    
    if orient=='horizontal':
        sns.set_context("paper", font_scale=1.5, rc={"lines.linewidth": 1})   
        if len(groups)<4:
            fig, axs = plt.subplots(1, len(groups), figsize=(len(groups)*6,2), constrained_layout=True)
        else:
            fig, axs = plt.subplots(2, m.ceil(len(groups)/2), figsize=(len(groups)*3,6), constrained_layout=True)
        
    if orient == 'vertical':
        sns.set_context("paper", font_scale=2, rc={"lines.linewidth": 2})   
        fig, axs = plt.subplots(len(groups), 1, figsize=(5,len(groups)*2.5), constrained_layout=True)
    return (params_heatmap,axs)

In [ ]:
def df_count(dfg1,**smooth):
    
    dfg1=dfg1.groupby(['zled','tled','inh_exp_id']).count()[var].reset_index()
    dfg1=dfg1.groupby(['zled','tled']).mean().reset_index()
    dfg1=dfg1.pivot(index='zled', columns='tled', values=var)
    dfg1=dfg1.fillna(0)
    im_filter='none'
    for arg in smooth.values():
            dfg1=heatmap_filter(dfg1,arg)
            im_filter=arg
    return dfg1,im_filter
        

In [ ]:
def df_mean(df,**smooth):
    dfg_mean=df.groupby(['zled','tled']).mean()[var].reset_index()
    #df_count=dfgn.groupby(['zled','tled','inh_exp_id']).count()['pid'].reset_index()
    #dfg_mean['count']=df_count.groupby(['zled','tled']).mean().reset_index()['pid']
    #dfg1['mask']=(dfg1['count']>count_threshold)|(dfg1['zled']>c_thr)
    #dfg1=pd.concat([dfg_mean,df_count],axis=1)
    #dfg_mean.loc[((dfg_mean['count']<count_threshold)&(dfg_mean['zled']>4)),var]=np.nan
    dfg1=dfg_mean.pivot(index='zled', columns='tled', values=var)
    dfg1=dfg1.fillna(min_col) #Kan censurera helt om man tar bort denna
    #dfg1=dfg1.fillna(0)
    im_filter='none'
    for arg in smooth.values():
        dfg1=heatmap_filter(dfg1,arg)
        im_filter=arg
    return dfg1,im_filter
        

In [1]:
#---------------------------------------------------------------------------
#Function for heatmap with count as color

def heatmap_count(dfg,**smooth):#max_col,min_col,,groups,orient,var,
     
    inside=dfg.inside_injury.unique()
    #dfg=led_bins(dfg)
    params_heatmap,axs=def_heatmap_params()#max_col,min_col,groups,orient
    
    for n,ax in enumerate(axs):
        
        dfg1=dfg.loc[(dfg.inh==groups[n])]
        dfg1,im_filter=df_count(dfg1,**smooth)
        
        #dfg1[dfg1.iloc[:,:]<count_threshold]=min_col
        plot_heatmap(dfg1,n,ax,params_heatmap,)
        
    
    save_fig(var,'heatmap_count')
    plt.show()
     
    #return dfg1
    #plt.colorbar((0,80))
    

#---------------------------------------------------------------------------    

In [1]:
#---------------------------------------------------------------------------
#Function for heatmap with variable mean as color
    
def heatmap_mean(dfg,**smooth):#,center_color,max_col,min_col,groups,var,orient#groups=inh_order count_thr=0
    
    inside=dfg.inside_injury.unique()
    
    params_heatmap,axs=def_heatmap_params()
    
    for n,ax in enumerate(axs):
        
        dfgn=dfg.loc[(dfg.inh==groups[n])]       
        dfg_mean,im_filter=df_mean(dfgn,**smooth)
        
        #dfg1=dfg1.where(count_mask, min_col)
        if count_thr>0:
            dfg_count,im_filter=df_count(dfgn,**smooth)
            #count_mask=dfg_count.iloc[:,:]<count_threshold
            #dfg1=dfg1.groupby(['zled','tled','inh_exp_id']).mean()[var].reset_index()
            dfg_count=dfg_count.reindex_like(dfg_mean)
            dfg_mean[dfg_count.iloc[:,:]<count_thr]=min_col
        
        plot_heatmap(dfg_mean,n,ax,params_heatmap)
    
    save_fig(var,'heatmap_mean')
    plt.show()
    

    
#---------------------------------------------------------------------------    

In [2]:
### OBS! EJ KLAR! SKA GE PROPORTION SOM UPPFYLLER ETT VISS VILLKOR EX TRACKNR 1-2 ELLER CONTRACTILE OSV
def heatmap_perc(dfg,dfg_c,**smooth):#,center_color
        
    count_threshold=10
    
    inside=dfg.inside_injury.unique()
    params_heatmap,axs=def_heatmap_params()
    
    #dfg=led_bins(dfg)
    #dfg_c=led_bins(dfg_c)
    
    for n,ax in enumerate(axs):
        
        dfg1=dfg.loc[(dfg.inh==groups[n])]
        dfg1_c=dfg_c.loc[(dfg_c.inh==groups[n])]
        
        dfg1=dfg1.groupby(['zled','tled','inh_exp_id']).count()[var]#.reset_index()
        dfg1_c=dfg1_c.groupby(['zled','tled','inh_exp_id']).count()[var]#.reset_index()
        
        dfg1i=dfg1.reset_index()
        df_count=dfg1i.groupby(['zled','tled']).mean().reset_index()
        #print(df_count)
        dfg1=dfg1_c/dfg1#dfg1_c[var]/dfg1[var]
        #dfg1=dfg1.reset_index()
        #print(dfg1.head())
        dfg1=dfg1.groupby(['zled','tled']).mean().reset_index()
        dfg1['count']=df_count[var]
        #dfg1['mask']=(dfg1['count']>count_threshold)|(dfg1['zled']>4)
        #dfg1.loc[((dfg1['count']<count_threshold)&(dfg1['zled']>4)),var]=np.nan
        
        #df_mask=dfg1.pivot(index='zled', columns='tled', values='mask')
        #mask=(df_mask['count']<count_threshold)&(df_mask['zled']>4)
        
        dfg1=dfg1.pivot(index='zled', columns='tled', values=var)
        dfg1=dfg1.fillna(min_col)
        
        
        for arg in smooth.values():
            dfg1=heatmap_filter(dfg1,arg)
            #print(dfg1)
        plot_heatmap(dfg1,n,ax,params_heatmap)
        #df_mask1=dfg1.copy()
        #df_mask1.loc[:,:]=min_col
        #print(df_mask)
        #print()
        #df_mask=(df_mask | dfg1>df_mask1)
        #df_mask=~df_mask
        #dfg1=dfg1[df_mask]#=min_col
        #dfg1=dfg1.fillna(min_col)
    save_fig(var,'heatmap_perc')
    plt.show()
   
    #plt.colorbar((0,80))
#---------------------------------------------------------------------------    

In [ ]:
def plot_heatmap(dfg1,n,ax,params_heatmap):
    sns.set_context("notebook", font_scale=2, rc={"lines.linewidth": 2})   
    if orient=='horizontal':
        horizontal_heatmap(dfg1,n,ax,params_heatmap) 
                
    if orient == 'vertical':
        vertical_heatmap(dfg1,n,ax,params_heatmap)
            
    
    

In [3]:
def vertical_heatmap(dfg1,n,ax,params_heatmap):
    if n == (len(groups)-1):     
        sns.heatmap(dfg1, yticklabels=9, cbar=True,ax=ax,**params_heatmap)
        ax.set_xlabel('Time (s)',fontsize=14)
                
    else:
        sns.heatmap(dfg1, yticklabels=9,cbar=False,ax=ax,**params_heatmap)
    ax.invert_yaxis()
    ax.set_ylabel('Height ($\mu$m)',fontsize=14)
    ax.set_ylabel('')
    ax.set_xlabel('')
            
    #ax.set_title(groups[n])
    ax.annotate(groups[n], xy=(3, 1), color='white',  xycoords='data',
    xytext=(0.8, 0.95), textcoords='axes fraction',
    horizontalalignment='right', verticalalignment='top',fontsize=20)
            

In [ ]:
def horizontal_heatmap(dfg1,n,ax,params_heatmap):
    if n == (len(groups)-1):                           
        sns.heatmap(dfg1, yticklabels=8,cbar=True,ax=ax,**params_heatmap)
                    
                    
                    #ax.invert_yaxis()
                    
    else:  
        sns.heatmap(dfg1, yticklabels=False,cbar=False,ax=ax,**params_heatmap)      
    ax.invert_yaxis()
    ax.yaxis.tick_right() 
    ax.set_ylabel('')
    ax.set_xlabel('')
    #ax.set_title(groups[n])
    ax.annotate(groups[n], xy=(3, 1), color='white',  xycoords='data',
    xytext=(0.8, 0.95), textcoords='axes fraction',
    horizontalalignment='right', verticalalignment='top',fontsize=20)

In [ ]:
def heatmap_filter(dfg1,smooth):
#------------------------------NEW PART WITH FILTER------------------------------------            
    dfg_n=dfg1.to_numpy()
    if smooth == 'uniform':
        dfg_n=ndimage.uniform_filter(dfg_n, size=2)
    if smooth == 'gauss':
        dfg_n=ndimage.gaussian_filter(dfg_n, sigma=2)#sigma=2
    if smooth == 'gauss1D':
        dfg_n=ndimage.gaussian_filter1d(dfg_n, sigma=2)
    
    #gaussian_filter1d(x, 3)
    
    #dfg_n=ndimage.uniform_filter(dfg_n, size=2)
    dfg1.loc[:,:]=dfg_n
    #dfg1=dfg1.fillna(0)
    return dfg1
#---------------------------------------------------------------------------            